In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("spark-sql")
         .getOrCreate())
sc = spark.sparkContext

24/07/21 16:00:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
file = "../data/20240721_23시_Youtube_인기급상승_동영상_videos.json"
video = spark.read.option("multiLine", "true").json(file)
video.printSchema()

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- contentDetails: struct (nullable = true)
 |    |    |    |-- caption: string (nullable = true)
 |    |    |    |-- definition: string (nullable = true)
 |    |    |    |-- dimension: string (nullable = true)
 |    |    |    |-- duration: string (nullable = true)
 |    |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |    |-- projection: string (nullable = true)
 |    |    |    |-- regionRestriction: struct (nullable = true)
 |    |    |    |    |-- allowed: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- categoryId: string (nullable = true)
 |    |    |    |-- channelId: string (nullab

In [11]:
video.show(5)

+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|                etag|               items|                kind|nextPageToken| pageInfo|            updatedt|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|ezNKxKzjJIYQe5Tqp...|[{{false, hd, 2d,...|youtube#videoList...|       CDIQAA|{50, 200}|[{2024-07-21 23:1...|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+



In [18]:
video.select('items').show(3)

+--------------------+
|               items|
+--------------------+
|[{{false, hd, 2d,...|
+--------------------+



In [3]:
from pyspark.sql.functions import explode

df_video = video.select(
               explode("items").alias("item"),
               "item.snippet.publishedAt",
               "item.snippet.channelId",
               "item.snippet.channelTitle",
               "item.id",
               "item.snippet.title",
               "item.snippet.tags",
               "item.snippet.categoryId",
               "item.contentDetails.duration",
               "item.statistics.viewCount",
               "item.statistics.likeCount",
               "item.statistics.favoriteCount",
               "item.statistics.commentCount"
           )
df_video = df_video.drop("item")
df_video.show(3)

+--------------------+--------------------+-------------------+-----------+-------------------------------+---------------------------------+----------+--------+---------+---------+-------------+------------+
|         publishedAt|           channelId|       channelTitle|         id|                          title|                             tags|categoryId|duration|viewCount|likeCount|favoriteCount|commentCount|
+--------------------+--------------------+-------------------+-----------+-------------------------------+---------------------------------+----------+--------+---------+---------+-------------+------------+
|2024-07-20T14:04:00Z|UC6erIDuvbOaAO-OT...|          SBS Drama|he86o7RI7dU|[#습포이드] 복수를 위한 이혼...|[스브스드라마, 굿파트너 장나라...|        24|PT29M20S|   934699|     8024|            0|         653|
|2024-07-20T09:00:39Z|UChgOgRNxrtCcVARD...|짧은대본 ShortPaper|pBCCrmVYPss|                 연애 온도 차이|          [드라마, web drama, ...|        24|PT12M59S|   617400|    12121|            0|      

In [35]:
df.show(1, False)

+--------------------+------------------------+------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+---------+---------+-------------+------------+
|publishedAt         |channelId               |channelTitle|id         |title                                                                                                                                                |tags                                                                                                                                                     |categoryId|duration|viewCount|likeCount|favoriteCount|commentCount|
+--------------------+------------------------+------------+-----------+----------------------------------------

In [4]:
files = "../data/comments_data/*.json"
comment = spark.read.option("multiLine", "true").json(files)
comment.printSchema()

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- canReply: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- isPublic: boolean (nullable = true)
 |    |    |    |-- topLevelComment: struct (nullable = true)
 |    |    |    |    |-- etag: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- kind: string (nullable = true)
 |    |    |    |    |-- snippet: struct (nullable = true)
 |    |    |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |    |    |-- authorChannelUrl: string (nullable = true)
 |    |    |    |    | 

In [5]:
df_comment = comment.select(
               explode("items").alias("item"),
               "item.snippet.videoId",
               "item.snippet.topLevelComment.snippet.textOriginal",
               "item.snippet.topLevelComment.snippet.authorDisplayName",
               "item.snippet.topLevelComment.snippet.likeCount",
               "item.snippet.topLevelComment.snippet.publishedAt"
           )
df_comment = df_comment.drop("item")
df_comment.show(3)

+-----------+----------------------------------+-----------------+---------+--------------------+
|    videoId|                      textOriginal|authorDisplayName|likeCount|         publishedAt|
+-----------+----------------------------------+-----------------+---------+--------------------+
|3ekhZlE5fq0|    📣 헤븐 그 자체인 프로 아이...|           @ch117|      480|2024-07-18T13:02:38Z|
|3ekhZlE5fq0| 태연은 헤메에 따라 분위기 다양...| @user-rh1ju6oy9u|      112|2024-07-19T04:54:33Z|
|9SA6degN604|화려하고 예쁘장하게 생겨서 제일...|         @souuppu|      367|2024-07-18T19:13:20Z|
+-----------+----------------------------------+-----------------+---------+--------------------+
only showing top 3 rows



In [41]:
import pyspark.sql.functions as F

df_comment.agg(F.countDistinct('videoId')).show(5, False)

+--------------+
|count(videoId)|
+--------------+
|50            |
+--------------+



In [7]:
from pyspark.sql.functions import count

comments_counts = df_comment.groupBy("videoId").agg(count("*").alias("count"))
comments_counts.show()

+-----------+-----+
|    videoId|count|
+-----------+-----+
|pf7QYivvs4c|    2|
|H5_KjoIUXOQ|    2|
|uOCY-KYU6qY|    2|
|u6bkTR_deNM|    2|
|pEE9cygiBTE|    2|
|Hjv_PARH0hs|    2|
|UcibwCZ-ta0|    2|
|W4pbL8e97J4|    2|
|EKQT8IP1TeU|    2|
|4SNThp0YiU4|    2|
|ieZi8Q-eVSQ|    2|
|9BPqIFTQ0U8|    2|
|XEUNO49sRH4|    2|
|XKIRUOlNdpo|    2|
|Av9DvtlJ9_M|    2|
|3ekhZlE5fq0|    2|
|9HU9t8VFxR4|    2|
|0P0aQreFs8w|    2|
|7FBFjSOKB0Y|    2|
|AOiZ-IekJg0|    2|
+-----------+-----+
only showing top 20 rows



In [9]:
df_video.createOrReplaceTempView("videos")
df_comment.createOrReplaceTempView("comments")

In [12]:
df_join = spark.sql("SELECT * FROM videos as A LEFT JOIN comments as B ON A.id = B.videoId")
df_join.show(3)

+--------------------+--------------------+-------------------+-----------+-------------------------------+---------------------------------+----------+--------+---------+---------+-------------+------------+-----------+---------------------------------+-----------------+---------+--------------------+
|         publishedAt|           channelId|       channelTitle|         id|                          title|                             tags|categoryId|duration|viewCount|likeCount|favoriteCount|commentCount|    videoId|                     textOriginal|authorDisplayName|likeCount|         publishedAt|
+--------------------+--------------------+-------------------+-----------+-------------------------------+---------------------------------+----------+--------+---------+---------+-------------+------------+-----------+---------------------------------+-----------------+---------+--------------------+
|2024-07-20T14:04:00Z|UC6erIDuvbOaAO-OT...|          SBS Drama|he86o7RI7dU|[#습포이드] 복수를 위

In [20]:
spark.sql("SELECT categoryId, COUNT(*) as count FROM videos GROUP BY categoryId").show()

+----------+-----+
|categoryId|count|
+----------+-----+
|        22|   12|
|        17|    2|
|        26|    3|
|        19|    3|
|        23|    5|
|        24|   17|
|         1|    1|
|        20|    3|
|        10|    4|
+----------+-----+



## 댓글 분석

### 한국어 / 영어 분리

In [49]:
language = spark.sql("""
        SELECT videoId, textOriginal,
          CASE
            WHEN REGEXP_REPLACE(textOriginal, '[^가-힣]', '') <> '' THEN '한국어'
            WHEN REGEXP_REPLACE(textOriginal, '[^a-zA-Z]', '') <> '' THEN '영어'
            ELSE '기타'
          END AS language
        FROM comments
        """)
language.show()

+-----------+----------------------------------+--------+
|    videoId|                      textOriginal|language|
+-----------+----------------------------------+--------+
|3ekhZlE5fq0|    📣 헤븐 그 자체인 프로 아이...|  한국어|
|3ekhZlE5fq0| 태연은 헤메에 따라 분위기 다양...|  한국어|
|9SA6degN604|화려하고 예쁘장하게 생겨서 제일...|  한국어|
|9SA6degN604|  기꺼이 8번출구 아저씨가 되어 ...|  한국어|
|nAHIo8o9sMQ|  🕸본인등판 데드풀과 울버린 편...|  한국어|
|nAHIo8o9sMQ| 아니 할리우드 대표 배우들이 자...|  한국어|
|CJ7uX5sXfuc| 이 링크로 스픽에 들어가시면 구...|  한국어|
|CJ7uX5sXfuc|가위 하나로 만든다길래 부엌에서...|  한국어|
|pEE9cygiBTE|            00:46⚡#NCT127 #삐그...|  한국어|
|pEE9cygiBTE|              No one will under...|    영어|
|qcKnIYckJHM|    여러분, 안녕하세요.

먼저...|  한국어|
|qcKnIYckJHM|  참피디 리스펙하고 대단한점

1...|  한국어|
|QfpTHAfe1ys|  캔디님들🍬오늘은 바삭한 '배추...|  한국어|
|QfpTHAfe1ys|최화정님이 부러운건 럭셔리한 삶...|  한국어|
|ospP1kfxWIo|   와 소고기 2만원 어치 사서 미...|  한국어|
|ospP1kfxWIo|    와 주연님🥹 저 댓글 진짜 잘...|  한국어|
|kC4Rxnk-c4c|   🧊먹어볼래 X 더미식 초계국수...|  한국어|
|kC4Rxnk-c4c| 광고주가 영상보고 진짜 만족하겠다|  한국어|
|pBCCrmVYPss|         멤버십 가입 - https://.

In [36]:
df_comment.select("textOriginal").collect()[8]

Row(textOriginal='00:46⚡#NCT127 #삐그덕 #Walk 킬포인트 삐그덕 댄스\r\n02:04⚡#NCT127 #삐그덕 #Walk 킬포인트 삐그덕 댄스 + 걸어 Walk 댄스\r\n\r\n 🔊1theKILLPO - NCT 127 킬포인트 "삐그덕 댄스 + 걸어 Walk 댄스" 이벤트\r\nⓛ 1theK Originals에서 \'1theKILLPO NCT 127 편\'을 시청하기\r\n② 킬포인트 “삐그덕 댄스 + 걸어 Walk 댄스” 부분의 타임코드(0:46, 2:04)와 본인이 느낀 삐그덕 댄스 + 걸어 Walk 댄스의 매력을 댓글로 남기기!\r\n③ 1theK Originals 채널 구독하기!\r\n✔참여기간 : 7/17 ~ 7/24\r\n✔추첨을 통해 NCT 127의 친필싸인이 담긴 폴라로이드를 드립니다🥰(당첨자 개별연락)\r\n✔타인의 글을 도용한 경우 당첨이 취소됩니다.\r\n\r\n🔊K-POP Wonderland, 1theK 👇\r\nSubscribe Now!👉https://www.youtube.com/1theKOriginals?sub_confirmation=1')

In [37]:
df_comment.select("textOriginal").collect()[9]

Row(textOriginal="No one will understand my obsession with the way Mark throws his head back and moves his shoulders when he's particularly feeling the choreo. Like what he's doing here💗❤\u200d🔥")

In [51]:
import pyspark.sql.functions as F

language.groupby('videoId', 'language').agg(count('language')).show()

+-----------+--------+---------------+
|    videoId|language|count(language)|
+-----------+--------+---------------+
|EKQT8IP1TeU|    영어|              2|
|uOCY-KYU6qY|  한국어|              2|
|pEE9cygiBTE|    영어|              1|
|NVDLUJa5dac|  한국어|              2|
|Av9DvtlJ9_M|    영어|              1|
|0_1x0s75cQ8|  한국어|              2|
|pYru6V3zmis|  한국어|              2|
|pEE9cygiBTE|  한국어|              1|
|coN9gSUo1Nk|  한국어|              2|
|7FBFjSOKB0Y|  한국어|              2|
|pf7QYivvs4c|    영어|              1|
|Hjv_PARH0hs|  한국어|              2|
|9HU9t8VFxR4|  한국어|              2|
|DEwrY0cm6-0|  한국어|              2|
|ieZi8Q-eVSQ|  한국어|              2|
|MmQbHRykuGQ|  한국어|              2|
|pUIAPXUUOOE|  한국어|              2|
|CJ7uX5sXfuc|  한국어|              2|
|qcKnIYckJHM|  한국어|              2|
|YxgzKc0Src8|  한국어|              2|
+-----------+--------+---------------+
only showing top 20 rows

